In [155]:
# make sure we have the most current version of github repo
%cd Wetland_Unet/azure/scv
!git pull
%cd ..

/mnt/batch/tasks/shared/LS_root/mounts/clusters/wetlandsbasiccpu/code/Users/mevans/Satellite_ComputerVision
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 20 (delta 16), reused 19 (delta 15), pack-reused 0
Unpacking objects: 100% (20/20), 5.52 KiB | 2.00 KiB/s, done.
From https://github.com/mjevans26/Satellite_ComputerVision
   164ff20..ce176df  master     -> origin/master
Updating 164ff20..ce176df
Fast-forward
 .spyproject/workspace.ini |   2 +-
 azure/retrain_wetland.py  | 232 ++++++++++++++++++++++++++++++++++++++++++++++
 azure/train_acd.py        |   4 +-
 azure/train_wetland.py    |  48 +++++-----
 utils/model_tools.py      |  18 ++--
 utils/prediction_tools.py |   4 +-
 6 files changed, 276 insertions(+), 32 deletions(-)
 create mode 100644 azure/retrain_wetland.py
/mnt/batch/tasks/shared/LS_root/mounts/clusters/wetlandsbasiccpu/code/Users/mevans


In [1]:
from azureml.tensorboard import Tensorboard
from azureml.core import Experiment, Environment, Workspace, Datastore, Dataset, Model, ScriptRunConfig
import os
import glob
# get the current workspace
ws = Workspace.from_config()

In [2]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'wetlanddatastore')

train_path = (datastore, 'summerReplicated/training/')
eval_path = (datastore, 'summerReplicated/eval/')

test_path = (datastore, 'summerReplicated/predict/test_aoi_Kumar_LES')

train_dataset = Dataset.File.from_files(path = [train_path])
eval_dataset = Dataset.File.from_files(path = [eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [3]:
# FInd the run corresponding to the model we want to register
run_id = 'wetland-unet-les_1648603447_6a7d97c7'

run = ws.get_run(run_id)

run.tags

{'splits': 'None',
 'one_hot': 'geomorphons',
 'model': 'Unet',
 'normalization': 'pixel',
 'features': 'full',
 'epochs': '150-200',
 'weight': '5',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}'}

In [4]:
model_name = 'wetland-unet-full-LES'

In [6]:
# if we haven't already registered our model
model = run.register_model(model_name= model_name,
                           tags=run.tags,
                           description = 'UNET model delineating wetlands using Sentinel-2, NAIP data from LES',
                           model_path='outputs/',
                           model_framework = 'Tensorflow',
                           model_framework_version= '2.0',
                           datasets = [('training', train_dataset), ('evaluation', eval_dataset), ('testing', test_dataset)])
print(model.name, model.id, model.version, sep='\t')

wetland-unet-basic-LES	wetland-unet-basic-LES:2	2


In [6]:
model_id = model_name
model = Model(ws, model_id)
model.version
model.get_model_path(model_id, _workspace = ws)

# model_path = model.get_model_path(model_id, _workspace = ws)
# model_path

'azureml-models/wetland-unet-full-LES/3/outputs'

In [7]:
model.run_id

'wetland-unet-les_1648603447_6a7d97c7'

In [ ]:
Model.get_model_path(model_id, _workspace = ws)

In [8]:
# use the azure folder as our script folder
script_folder = 'Wetland_UNet/azure'
script_file = 'retrain_wetland.py'

In [9]:
# get our environment
envs = Environment.list(workspace = ws)
env = envs.get('wetland-training')

In [10]:
experiment_name = 'wetland-unet-les'
exp = Experiment(workspace = ws, name = experiment_name)

## Dataset Metadata
We need to calculate, at a minimum, the size of our training dataset to pass to the re-training script

In [29]:
import sys
sys.path.append('/home/azureuser/cloudfiles/code/Users/mevans/Wetland_UNet/azure/scv')

In [30]:
from utils.model_tools import get_binary_model, make_confusion_matrix, weighted_bce
from utils.processing import get_training_dataset, get_eval_dataset
from utils.prediction_tools import make_pred_dataset, callback_predictions, plot_to_image
import tensorflow as tf
from os.path import join

In [31]:
# Define some global variabes

# specify surface layers
lidar = ['lidar_intensity']
geomorphon = ["geomorphons"]

# Specify inputs (Sentinel bands) to the model
opticalBands = ['B3', 'B4', 'B5', 'B6']
thermalBands = ['B8', 'B11', 'B12']
senBands = opticalBands + thermalBands

# get band names for three seasons
seasonalBands = [[band+'_summer', band + '_fall', band + '_spring'] for band in senBands]


# specify NAIP bands
naipBands = ['R', 'G', 'B', 'N']

BANDS = [item for sublist in seasonalBands for item in sublist] + naipBands
MORPHS = lidar + geomorphon
RESPONSE = 'wetland'
FEATURES = BANDS + MORPHS+ [RESPONSE]
print(FEATURES)
# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))



['B3_summer', 'B3_fall', 'B3_spring', 'B4_summer', 'B4_fall', 'B4_spring', 'B5_summer', 'B5_fall', 'B5_spring', 'B6_summer', 'B6_fall', 'B6_spring', 'B8_summer', 'B8_fall', 'B8_spring', 'B11_summer', 'B11_fall', 'B11_spring', 'B12_summer', 'B12_fall', 'B12_spring', 'R', 'G', 'B', 'N', 'lidar_intensity', 'geomorphons', 'wetland']


In [102]:
# Create mountcontext and mount the dataset
with train_dataset.mount() as train_mount:

    # Get the mount point
    dataset_mount_folder = train_mount.mount_point
    print(dataset_mount_folder)

    train_files = []

    for root, dirs, files in os.walk(dataset_mount_folder):
        for f in files:
            if '256wetlandLES' in f:
                train_files.append(join(root, f))

    training = get_training_dataset(
        files = train_files,
        ftDict = FEATURES_DICT,
        features = naipBands,#BANDS+MORPHS, # reading data can be memory intensive. for the purpose of calculating class weights, we don't need all the predictor variables
        response = RESPONSE,
        buff = 1,
        batch = 1,
        repeat = False,
        one_hot = None,
        splits = None)
    
    i = 0
    # # calculate some summary statistics used in model training
    iterator = iter(training)
    wetlandPix = 0
    nonPix = 0
    for example in iterator:
        ones = tf.reduce_sum(example[1])
        zeroes = (256*256) - ones
        wetlandPix += ones
        nonPix += zeroes
        i += 1
    # m = get_binary_model(depth = len(naipBands), optim = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999), loss = 'mse', mets = [tf.keras.metrics.categorical_accuracy], bias = None)
    # # preds = m.predict(example[0], verbose = 1)
    # train_con_mat = make_confusion_matrix(training, m)
    # classums = train_con_mat.sum(axis = 1)
    # BIAS = np.log(classums[1]/classums[0])
    # WEIGHT = classums[0]/classums[1]
    # TRAIN_SIZE = train_con_mat.sum()//(256*256)

    
    
print(i)

/tmp/tmpvmk3z1s2
Downloaded path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/summerReplicated/training/UNET_256wetlandLES10.tfrecord.gz is different from target path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/UNET_256wetlandLES10.tfrecord.gz
Downloaded path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/summerReplicated/training/UNET_256wetlandLES100.tfrecord.gz is different from target path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/UNET_256wetlandLES100.tfrecord.gz
Downloaded path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/summerReplicated/training/UNET_256wetlandLES1000.tfrecord.gz is different from target path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/UNET_256wetlandLES1000.tfrecord.gz
Downloaded path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/summerReplicated/training/UNET_256wetlandLES1005.tfrecord.gz is different from target path: /tmp/tmpe3whikq9/c51364d3-fc2f-4a51-8dc9-566f591b2c38/UNET_256wetlandL

In [109]:
WEIGHT = nonPix/wetlandPix
WEIGHT

<tf.Tensor: shape=(), dtype=float32, numpy=20.2953>

In [58]:
tf.reduce_min(example[0][:, 0:256, 0:256, 36])

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

## Experiment Setup

In [11]:
# define the compute target
ws.compute_targets
wetlandGPU = ws.compute_targets['wetlandsGPU']
wetlandDSVM = ws.compute_targets['wetland-dsvm']
wetlandDSVM2 = ws.compute_targets['wetland-dsvm2']

In [12]:
# define the command line arguments to our training sript
weight = 2
args = [
    '--train_data', train_dataset.as_mount(),
    '--eval_data', eval_dataset.as_mount(),
    '--subset', '256wetlandLES',
    '--test_data', test_dataset.as_mount(),
    '--model_id', model_id,
    '--weight', weight, 
    '--bias', 0,
    '-lr', 0.0005,
    '--epochs', 50,
    '--batch', 16,
    '--size', 1788,#7207, #14225,
    '--kernel_size', 256,
    '--response', 'wetland']

In [15]:
#  run the training job
features = 'full'
src = ScriptRunConfig(source_directory=script_folder,
                      script=script_file, 
                      arguments=args,
                      compute_target=wetlandDSVM2,
                      environment=env)
run = exp.submit(config=src, tags = dict({'splits':'None', 'one_hot':'geomorphons', 'model':'Unet', 'normalization':'pixel', 'features':f'{features}', 'epochs':'200-250', 'weight':f'{weight}'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
wetland-unet-les,wetland-unet-les_1648748275_776a08c2,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
#  run the training job
features = 'basic'
src = ScriptRunConfig(source_directory=script_folder,
                      script=script_file.format(features), 
                      arguments=args,
                      compute_target= wetlandGPU,
                      environment=env)
run = exp.submit(config=src, tags = dict({'splits':'None', 'one_hot':'none', 'model':'Unet', 'normalization':'pixel', 'features':f'{features}', 'epochs':'150-200', 'weight':'2'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
wetland-unet-les,wetland-unet-les_1648737274_0520f70d,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation
